<a href="https://colab.research.google.com/github/LucaNavarra/MC3/blob/main/MC3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importo le librerie e monto google drive

In [ ]:
import scipy.io as sio
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import shutil
import copy
import random
from random import shuffle
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Check device

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
#rendere l'esecuzione deterministica
torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#Esempio visualizzazione immagine e divisione train/val



In [ ]:
#settare path delle immagini
imgs_path = '/content/drive/MyDrive/uni-na-machine-learning-23-24-ii-sem-contest-3/'


#Test visualizzazione immagine dal train set
img1 = np.load(imgs_path+'train/train/benign/mcf10aCdExposed-01/mcf10aCdExposed-01_croppedCell_21.npy')
img2 = np.load(imgs_path + 'train/train/malignant/mcf7CdExposed-04/mcf7CdExposed-04_croppedCell_12.npy')

print(img1.shape)
plt.figure()
plt.imshow(img1)

print(img2.shape)
plt.figure()
plt.imshow(img2)

#Dalle shape vedo che le foto sono tutte quadrate ma la dimensione è variabile





#def split_train_val(root, classes):
#  if "val" not in os.listdir(root):
#    os.mkdir(root + '/val')
#    for c in classes:
#      os.mkdir(root + '/val/' + c)
#
#  for c in classes:
#    list_folder = os.listdir(root + '/train' + '/' + c)
#    for folder in list_folder:
#        listofFiles = os.listdir(root + '/train' + '/' + c + '/' + folder)
#        for file in listofFiles:
#          if random.random() > 0.7:
#            if folder not in os.listdir(root + '/val/' + c):
#              os.mkdir(root + '/val/' + c + '/' + folder)
#             shutil.move(root + 'train' + '/' + c + '/' + folder + '/' + file, root + 'val' + '/' + c + '/' + folder + '/' + file)

#classes = ['benign', 'malignant']
#split_train_val(imgs_path + 'train', classes)


In [ ]:
class My_DatasetFolder(Dataset):
    def __init__(self, root, transform, is_valid_file, list_classes):
        self.root = root
        self.transform = transform
        self.is_valid_file = is_valid_file
        self.list_classes = list_classes
        self.samples = self.__get_samples()

    def __len__(self):
        return len(self.samples)

    def __get_samples(self):
        ListFiles = []
        for c in self.list_classes:
            list_folder = os.listdir(self.root + '/' + c)
            for folder in list_folder:
                listofFiles = os.listdir(self.root + '/' + c + '/' + folder)
                for file in listofFiles:
                    if self.is_valid_file(self.root + '/' + c + '/' + folder + '/' + file):
                        ListFiles.append((self.root + '/' + c + '/' + folder + '/' + file, self.list_classes.index(c)))
        return ListFiles

    def loader_fc(self, path):
        img = np.load(path)
        return img

    def __getitem__(self, index: int):
        path, target = self.samples[index]
        sample = self.loader_fc(path)
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target

# Definizione della trasformazione dei dati
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

# Definizione dei set di dati utilizzando la tua classe My_DatasetFolder
train_dataset = My_DatasetFolder(root=imgs_path + 'train/train',
                                 transform=data_transforms['train'],
                                 is_valid_file=lambda path: path.endswith('.npy'),
                                 list_classes=['benign', 'malignant'])

val_dataset = My_DatasetFolder(root=imgs_path + 'train/val',
                               transform=data_transforms['validation'],
                               is_valid_file=lambda path: path.endswith('.npy'),
                               list_classes=['benign', 'malignant'])

# Definizione dei DataLoader
train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=32,
                                               shuffle=True,
                                               num_workers=2)

val_dataloader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=32,
                                             shuffle=False,
                                             num_workers=2)

# Raggruppo dei DataLoader in un dizionario
dataloaders = {'train': train_dataloader, 'validation': val_dataloader}

# Raggruppo image datasets in un dizionario
image_datasets = {'train': train_dataset, 'validation': val_dataset}

'''
un esempio di utilizzo
is_valid_file = lambda path: path.endswith('.npy')

My_DatasetFolder(root = MyPath,  transform = data_transforms, is_valid_file = is_valid_file, list_classes = ['benign', 'malignant'])
'''

In [ ]:
#outfeatures_train =[]

model = models.resnet50(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)
'''
class_list_train = []
class_list_test = []
classes = ['benign', 'malignant']
data_dir = 'data'   #path dei dati

for c in classes:
  folder = os.listdir(data_dir + '/train/' + c)
  for f in folder:
    listImage = os.listdir(data_dir + '/train/' + c + '/' + f)
    for input in listImage:
      img = np.load(data_dir + '/train/' + c + '/' + f + '/' + input)
      img = data_transforms(img).type(torch.FloatTensor).to(device)
      img = img.unsqueeze_(0)
      outputs = model(img)
      outfeatures_train.append((np.squeeze(outputs.cpu().detach()).numpy()))
      class_list_train.append(classes.index(c))
'''

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [ ]:

def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                inputs = inputs.to(torch.float32)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase] )
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model


Addestramento

In [ ]:
model_trained = train_model(model, criterion, optimizer, num_epochs=3)


Salvo i pesi

In [ ]:
!mkdir models
!mkdir models/pytorch

In [ ]:
torch.save(model_trained.state_dict(), 'models/pytorch/weights.h5')

In [ ]:
model = models.resnet50(pretrained=False).to(device)
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2)).to(device)
model.load_state_dict(torch.load('models/pytorch/weights.h5'))